In [37]:
!pip install -qqq  langchain
!pip install -qqq  langchain-core
!pip install -qqq  langchain-community
!pip install -qqq 'crewai[tools]' --no-deps
!pip install -qqq  langchain-experimental

In [5]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
!gdown 1Q7GnaGpXxmrI2S7mxKmeFFqj8dotsxAM

Downloading...
From: https://drive.google.com/uc?id=1Q7GnaGpXxmrI2S7mxKmeFFqj8dotsxAM
To: /content/ds-salaries.csv
100% 210k/210k [00:00<00:00, 109MB/s]


In [7]:
df = pd.read_csv("/content/ds-salaries.csv")

df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [50]:
df.shape

(3755, 11)

In [52]:
# delete
df = df[df['company_location'] != 'IL']

In [53]:
df.shape

(3753, 11)

In [55]:
connection = sqlite3.connect("salaries.db")
df.to_sql(name="salaries", con=connection)

3753

In [63]:
cursor = connection.cursor()
cursor.execute("SELECT company_location FROM salaries ")
locations = cursor.fetchall()

print(locations)

[('ES',), ('US',), ('US',), ('CA',), ('CA',), ('US',), ('US',), ('CA',), ('CA',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('DE',), ('DE',), ('GB',), ('GB',), ('US',), ('US',), ('US',), ('US',), ('CA',), ('CA',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('NG',), ('NG',), ('IN',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('HK',), ('US',), ('DE',), ('IN',), ('NL',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('ES',), ('ES',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), ('US',), 

In [66]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

In [67]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

In [68]:
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("human", human)])

chain = prompt | llm
response = chain.invoke(
    {
        "text": "Which company has better models OpenAI or Anthropic? Respond with just the company name."
    }
)

print(response.content)

Anthropic


In [69]:
response.__dict__


{'content': 'Anthropic',
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 3,
   'prompt_tokens': 28,
   'total_tokens': 31,
   'completion_time': 0.009678948,
   'prompt_time': 0.000993151,
   'queue_time': 0.012741007,
   'total_time': 0.010672099},
  'model_name': 'llama3-70b-8192',
  'system_fingerprint': 'fp_c1a4bcec29',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-b0504649-6b6d-4a47-ba0e-4d183dd7a64a-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': None}

In [70]:
db = SQLDatabase.from_uri("sqlite:///salaries.db")

In [71]:
@tool("list_tables")
def list_tables(config: Dict[str, Any] = None) -> str:
    """List the available tables in the database."""
    db_tool = ListSQLDatabaseTool(db=db, config=config)  # Passing config if necessary
    return db_tool.invoke("")

In [72]:
list_tables.run()

Using Tool: list_tables


'salaries'

In [73]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

In [74]:
print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2023	SE	FT	Principal Data Scientist	80000	EUR	85847	ES	100	ES	L
1	2023	MI	CT	ML Engineer	30000	USD	30000	US	100	US	S
2	2023	MI	CT	ML Engineer	25500	USD	25500	US	100	US	S
*/


In [75]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [78]:
execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 2023, 'SE', 'FT', 'Principal Data Scientist', 80000, 'EUR', 85847, 'ES', 100, 'ES', 'L'), (1, 2023, 'MI', 'CT', 'ML Engineer', 30000, 'USD', 30000, 'US', 100, 'US', 'S'), (2, 2023, 'MI', 'CT', 'ML Engineer', 25500, 'USD', 25500, 'US', 100, 'US', 'S'), (3, 2023, 'SE', 'FT', 'Data Scientist', 175000, 'USD', 175000, 'CA', 100, 'CA', 'M'), (4, 2023, 'SE', 'FT', 'Data Scientist', 120000, 'USD', 120000, 'CA', 100, 'CA', 'M')]"

In [79]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [80]:
check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'SELECT * FROM salaries WHERE salary > 10000 LIMIT 5'

In [81]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [82]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [83]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)


# Tasks

In [84]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [85]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [86]:

write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

# Crew

In [89]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="crew_2.log",
)

In [90]:
inputs = {
    "query": "Effects on salary (in USD) based on company location, size and employee experience"
}

result = crew.kickoff(inputs=inputs)

 [2024-10-15 09:35:21][DEBUG]: == Working Agent: Senior Database Developer
 [2024-10-15 09:35:21][INFO]: == Starting Task: Extract data that is required for the query Effects on salary (in USD) based on company location, size and employee experience.
 

salaries

 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool tables_schema accepts these inputs: tables_schema(tables: 'string') -      Input is a comma-separated list of tables, output is the schema and sample rows     for those tables. Be sure that the tables actually exist by calling `list_tables` first!     Example Input: table1, table2, table3     

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	compa

In [91]:
print(result)

### Executive Summary

The analysis of salary effects based on company location, size, and employee experience reveals that these factors significantly impact salary. Key findings include:

* **Company Location:** Countries such as the US, UK, Australia, and Canada offer the highest average salaries ($80,000-$150,000), while countries in Asia, South America, and Africa have lower average salaries ($10,000-$50,000).
* **Company Size:** Large companies tend to have higher average salaries than medium-sized and small companies, with a difference of $20,000-$30,000.
* **Employee Experience:** Employees with more experience tend to have higher average salaries than those with less experience, with a difference of $20,000-$30,000.
* **Interactions between Factors:** The interactions between company location, company size, and employee experience also significantly impact salary, with large companies in developed countries and employees with more experience in these companies tend to have hig

In [92]:
inputs = {
    "query": "How is the `Machine Learning Engineer` salary in USD is affected by remote positions"
}

result = crew.kickoff(inputs=inputs)

 [2024-10-15 09:39:44][DEBUG]: == Working Agent: Senior Database Developer
 [2024-10-15 09:39:44][INFO]: == Starting Task: Extract data that is required for the query How is the `Machine Learning Engineer` salary in USD is affected by remote positions.
 

salaries

 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool tables_schema accepts these inputs: tables_schema(tables: 'string') -      Input is a comma-separated list of tables, output is the schema and sample rows     for those tables. Be sure that the tables actually exist by calling `list_tables` first!     Example Input: table1, table2, table3     

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	com

In [93]:
print(result)

**Executive Summary**

### Key Findings

* The average salary for Machine Learning Engineers is approximately $173,411, with a median salary of $150,000 and a standard deviation of $63,419.
* There is a moderate positive correlation (r = 0.43) between remote work percentage and salary, indicating that higher remote work percentages tend to correspond to higher salaries.
* Machine Learning Engineers with full remote work (100%) tend to have higher average salaries ($184,615) compared to those with no remote work (0%) or partial remote work (50%).

### Insights

* Remote work opportunities may have a positive impact on the salaries of Machine Learning Engineers, but other factors such as experience, location, and company size may also play a role in determining salary trends.
* The significant range of salaries suggests that other factors beyond remote work percentage influence salary.
